<a href="https://colab.research.google.com/github/Vaycold/pytorch_tutorial/blob/main/%2305.torch%20autograd%EB%A5%BC%20%EC%82%AC%EC%9A%A9%ED%95%9C%20%EC%9E%90%EB%8F%99%20%EB%AF%B8%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Torch.autograd를 사용한 자동 미분

    : 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파
    : 매개변수(모델가중치)는 주어진 매개변수에 대한 손실 함수의 변화도에 따라 조정
    : torch.autograd라고 불리는 자동 미분 엔진이 내장


In [1]:
'''
입력 : x // 매개변수 : w,b // 
단일 계층 신경망 구성
'''

import torch
x = torch.ones(5) # INPUT TENSOR
y = torch.zeros(3) # EXPECTED OUTPUT
w = torch.randn(5,3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b

In [2]:
print(x)
print(y)
print('[w] \n'  ,w)
print(b)
print(z)

tensor([1., 1., 1., 1., 1.])
tensor([0., 0., 0.])
[w] 
 tensor([[ 0.0654,  0.7685,  1.3524],
        [ 0.9088,  1.0460,  0.8048],
        [ 0.2990, -0.2019, -0.9500],
        [ 0.6548,  1.1785,  0.5599],
        [-0.2507, -0.5031, -0.1404]], requires_grad=True)
tensor([-1.1980, -0.0105,  0.2240], requires_grad=True)
tensor([0.4794, 2.2774, 1.8507], grad_fn=<AddBackward0>)


In [3]:
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)
loss

tensor(1.7777, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [4]:
'''
이 신경망에서 w와 b는 최적화를 해야하는 매개변수
이러한 변수들에 대한 손실 함수의 변화도를 계산할 수 있어야
이를 위해서 해당 텐서에 requires_grad 속성을 설정
'''
pass

### 변화도 계산하기

    : 신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수를 계산하여야 함
    : 즉 x와 y의 일부 고정값에서 w에 대한 loss 미분값, b에 대한 loss미분값이 필요함
    : 이러한 도함수를 계산하기 위해 loss.backward()를 호출한 다음 w.grad와 b.grad에서 값을 가져옴

In [5]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2059, 0.3023, 0.2881],
        [0.2059, 0.3023, 0.2881],
        [0.2059, 0.3023, 0.2881],
        [0.2059, 0.3023, 0.2881],
        [0.2059, 0.3023, 0.2881]])
tensor([0.2059, 0.3023, 0.2881])


In [6]:
##############
# 연산 그래프의 잎 노드들 중 requires_grad 속성이 True로 설정된 노들의 grad속성만 구할 수있음
# 다른 모든 노드에서는 변화도가 유효하지 않음
# 성능상의 이유로, 주어진 그래프에서의 backward를 사용한 변화도 계산은 한번만 수행할 수 있음
# 만약 동일한 그래프에서 여러번의 backward 호출이 필요하면, backward 호출 시에 retrain_graph = True를 전달해야함

### 변화도 추적 멈추기

    : 기본적으로, requires_grad = True인 모든 텐서들은 연산기록을 추적하고 변화도 계산을 지원함
    : 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산이 필요한 경우에는 이러한 추적이나 지원이 필요없을 수 있음.
    : torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있음.

In [7]:
z = torch.matmul(x, w) +b
print(z.requires_grad)

with torch.no_grad() :
    z = torch.matmul(x,w) + b
print(z.requires_grad)

True
False


In [8]:
'''
변화도 추적을 멈춰야 하는 이유
 - 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시
  > 이는 사전학습된 신경망을 미세조정할때 매우 일반적인 시나리오
 - 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산속도 향상
 '''
pass

### 연산 그래프에 대한 추가 정보

    : 개념적으로 Autograd는 데이터(텐서)의 및 실행된 모든 연산들의 기록을 Function의 객체로 구성된 방향성 비순환 그래프에 저장
    : 이 방향성 비순환 그래프의 잎은 입력 텐서, 뿌리는 결과 텐서

    : 순전파 단계에서 autograd는 두 가지 작업 동시 수행
     - 요청된 연산을 수행하여 결과 텐서 계산
     - DAG에 연산의 변화도 기능을 유지

    : 역전파 단계는 DAG 뿌리에서 .backward() 가 호출될 때 시작
     - 각 .grad_fn으로부터 변화도를 계산
     - 각 텐서의 .grad 속성에 계산 결과를 쌓고
     - 연쇄 법칙을 사용하여, 모든 잎 텐서들까지 전파